# 割込み制御テストプログラム
- 表示回路２に割込み制御レジスタが正しく実装されているかを確認する
- 割込み処理が正しく実行されることを確認する

### [STEP-1] 定数定義
- レジスタアドレス設定

In [ ]:
# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

#FRAMEカウンタ
FRAME_CNT = 0

### [STEP-2] Overlay
- 表示回路２のコンフィギュレーション

In [ ]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-3] 割込みピンの確認
- 次の結果が得られればOK  
{'display_0/DSP_IRQ': {'controller': 'axi_intc_0',  
&nbsp;&nbsp;&nbsp;&nbsp;'index': 0,  
&nbsp;&nbsp;&nbsp;&nbsp;'fullpath': 'display_0/DSP_IRQ'},  
&nbsp;&nbsp;'axi_intc_0/intr': {'controller': 'axi_intc_0',  
&nbsp;&nbsp;&nbsp;&nbsp;'index': 0,  
&nbsp;&nbsp;&nbsp;&nbsp;'fullpath': 'axi_intc_0/intr'}}

In [ ]:
pl.interrupt_pins

### [STEP-4] 割込みの登録
- display_0はメモリマップされていないため、直接割込みの登録ができない
- 代わりに割込みコントローラーの割込みピン（axi_intc_0.intr）を登録する

In [ ]:
interrupt = pl.axi_intc_0.intr

### [STEP-5] 割込み待ち関数の定義
- await interrupt.wait() で割込みが発生するのを待つ
- 割込み発生後は割込みをクリア
- フレームカウントをカウントアップ

In [ ]:
async def wait_for_int() :
    global FRAME_CNT                          #グローバル変数
    await interrupt.wait()                    #割込み待ち
    pl.regbus_0.write(RA_DISPINT, 0x00000003) #割込みクリア
    FRAME_CNT = FRAME_CNT + 1                 #フレームカウンタインクリメント

### [STEP-6] メイン処理
- 割込みイネーブル設定 @ RA_DISPINTレジスタ
- asyncio ライブラリで割込み待ち関数をtaskに登録
- taskの処理待ち
- 10回処理を繰り返して終了
- FRAME_CNTが0～9で表示されればOK

In [ ]:
import asyncio

pl.regbus_0.write(RA_DISPINT, 0x00000001)  #割込みイネーブル
loop = asyncio.get_event_loop()

FRAME_CNT = 0

for I in range(10) :
    print("Frame Count= {}".format(FRAME_CNT))  #FRAME_CNTを表示
    task = loop.create_task(wait_for_int())     #割込み待ち関数を登録
    loop.run_until_complete(task)               #割込み待ち関数処理の終了待ち